In [116]:
import pandas as pd
import numpy as np

In [117]:
data_train = pd.read_csv("titanic/train.csv")
data_test = pd.read_csv("titanic/test.csv")
data_gs = pd.read_csv("titanic/gender_submission.csv")

In [118]:
data_train.shape

(891, 12)

In [119]:
y_pred = (data_train["Pclass"] == 1).astype(int)

In [120]:
def accuracy(y_pred, y):
    return (y_pred == y).astype(int).sum()/len(y)

In [121]:
accuracy(y_pred, data_train["Survived"])

0.6790123456790124

In [122]:
data_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [123]:
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [124]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [125]:
data_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [147]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [146]:
def create_feature(data_train):
    data_train["Pclass"] = data_train["Pclass"].apply(lambda x: str(x) + "Pclass")

    data_train["Embarked"] = data_train["Embarked"].fillna(data_train["Embarked"].value_counts().index[0])
    data_train["Embarked"] = data_train["Embarked"].apply(lambda x: str(x) + "Embarked")
    
    data_train["Age"] = data_train["Age"].fillna(data_train["Age"].mean())
    data_train["Fare"] = data_train["Fare"].fillna(data_train["Fare"].mean())
    data_train["SibSp"] = data_train["SibSp"].fillna(data_train["SibSp"].mean())
    data_train["Parch"] = data_train["Parch"].fillna(data_train["Parch"].mean())
    
    data_train["CabinFirst"] = data_train["Cabin"].str[:2]
    data_train["CabinFirst"] = data_train["CabinFirst"].fillna(data_train["CabinFirst"].value_counts().index[0])
    del data_train["Cabin"]
    
    data_train["Name_skobka"] = data_train.Name.apply(lambda x: "(" in str(x)).astype(int)
    data_train["Name_len"] = data_train.Name.str.len()
    data_train["Name_mr"] = data_train.Name.apply(lambda x: "mr" in str(x).lower()).astype(int)
    data_train["Name_mrs"] = data_train.Name.apply(lambda x: "mrs" in str(x).lower()).astype(int)
    data_train["Name_miss"] = data_train.Name.apply(lambda x: "miss" in str(x).lower()).astype(int)

    dict_name = data_train.Name.str.split(",", n=1, expand=True)[0].value_counts().to_dict()
    data_train["Name_count"] = data_train.Name.str.split(",", n=1, expand=True)[0].apply(lambda x: dict_name[x])
    del data_train["Name"]
    del data_train["Ticket"]
    del data_train["PassengerId"]
    
    data_train_proc = data_train.copy()
    
    enc = OneHotEncoder()
    category_columns = ["Sex", "Pclass", "Embarked", "CabinFirst"]
    enc.fit(data_train[category_columns])

    transform_data = enc.transform(data_train[category_columns]).toarray().T
    for j in range(len(category_columns)):
        for i, col in enumerate(enc.categories_[j]):
            data_train_proc[col] = transform_data[i]
    for col in category_columns:
        del data_train_proc[col]
        
    return data_train_proc[[el for el in data_train_proc.columns if el != "Survived"]], data_train_proc["Survived"]

In [128]:
X, y = create_feature(data_train)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [148]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.7985074626865671

In [144]:
sorted(list(zip(model.feature_importances_, model.feature_names_in_)), reverse=True)

[(0.11520610331133829, 'Fare'),
 (0.11300727062987588, 'Age'),
 (0.10212857298663608, 'Name_len'),
 (0.04637181364345665, 'Name_count'),
 (0.04560606114575909, 'male'),
 (0.04201567180136221, 'A2'),
 (0.041111433992318716, 'A1'),
 (0.04041839372598291, 'A6'),
 (0.038218161594470564, 'SibSp'),
 (0.035184182892801875, 'QEmbarked'),
 (0.030610334154280133, 'Name_mr'),
 (0.02784975192930436, 'Parch'),
 (0.027840627800816312, '1Pclass'),
 (0.02738294484896905, 'C9'),
 (0.026024785016966066, '2Pclass'),
 (0.025747950915969677, 'CEmbarked'),
 (0.024940435166475768, 'A5'),
 (0.020133166703474918, 'female'),
 (0.017769433026068808, 'Name_skobka'),
 (0.017364888171296108, 'B2'),
 (0.014226048563784694, 'A7'),
 (0.013507953203840685, 'F '),
 (0.011919481416837446, 'SEmbarked'),
 (0.010724733286544573, 'A3'),
 (0.010394105969270208, 'Name_mrs'),
 (0.009663482164508838, 'B1'),
 (0.00789443435081842, '3Pclass'),
 (0.007191970328955676, 'F2'),
 (0.005464668600852532, 'D2'),
 (0.005169193028519674, 'N

In [149]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.8022388059701493

In [152]:
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.8097014925373134

In [153]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.8134328358208955

In [155]:
model = RandomForestClassifier(n_estimators=10000, n_jobs=-1)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.8134328358208955

In [160]:
!pip3 install xgboost

     ---------------------------------------- 89.1/89.1 MB 3.6 MB/s eta 0:00:00


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [162]:
import xgboost as xgb

In [170]:
model = xgb.XGBClassifier(max_depth=10, n_estimators=500)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.7723880597014925

In [171]:
model = xgb.XGBClassifier(max_depth=5, n_estimators=500)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.7910447761194029

In [173]:
model = xgb.XGBClassifier(max_depth=3, n_estimators=500)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

0.7835820895522388

In [174]:
a = xgb.XGBClassifier()

In [175]:
a.n_estimators

100

In [100]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['1Pclass', '2Pclass', '3Pclass'], dtype=object),
 array(['CEmbarked', 'QEmbarked', 'SEmbarked'], dtype=object),
 array(['A1', 'A2', 'A3', 'A5', 'A6', 'A7', 'B1', 'B2', 'B3', 'B4', 'B5',
        'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7',
        'C8', 'C9', 'D', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D9',
        'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'F ', 'F2', 'F3',
        'F4', 'G6', 'T'], dtype=object)]

In [101]:
data_train_proc

,Survived,Age,SibSp,Parch,Fare,Name_skobka,Name_len,Name_mr,Name_mrs,Name_miss,...,E5,E6,E7,E8,F,F2,F3,F4,G6,T
0,0,22.000000,1,0,7.2500,0,23,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,38.000000,1,0,71.2833,1,51,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,26.000000,0,0,7.9250,0,22,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,35.000000,1,0,53.1000,1,44,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,35.000000,0,0,8.0500,0,24,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,0,0,13.0000,0,21,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
887,1,19.000000,0,0,30.0000,0,28,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0,29.699118,1,2,23.4500,0,40,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1,26.000000,0,0,30.0000,0,21,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
dict_name = data_train.Name.str.split(",", n=1, expand=True)[0].value_counts().to_dict()
data_train.Name.str.split(",", n=1, expand=True)[0].apply(lambda x: dict_name[x])

0      2
1      1
2      1
3      2
4      2
      ..
886    1
887    3
888    2
889    1
890    1
Name: 0, Length: 891, dtype: int64

In [42]:
data_train.Name.apply(lambda x: "(" in str(x)).astype(int)

0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    0
888    0
889    0
890    0
Name: Name, Length: 891, dtype: int32

In [41]:
data_train.Name.unique()

array(['Braund, Mr. Owen Harris',
       'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
       'Heikkinen, Miss. Laina',
       'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
       'Allen, Mr. William Henry', 'Moran, Mr. James',
       'McCarthy, Mr. Timothy J', 'Palsson, Master. Gosta Leonard',
       'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
       'Nasser, Mrs. Nicholas (Adele Achem)',
       'Sandstrom, Miss. Marguerite Rut', 'Bonnell, Miss. Elizabeth',
       'Saundercock, Mr. William Henry', 'Andersson, Mr. Anders Johan',
       'Vestrom, Miss. Hulda Amanda Adolfina',
       'Hewlett, Mrs. (Mary D Kingcome) ', 'Rice, Master. Eugene',
       'Williams, Mr. Charles Eugene',
       'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',
       'Masselmani, Mrs. Fatima', 'Fynney, Mr. Joseph J',
       'Beesley, Mr. Lawrence', 'McGowan, Miss. Anna "Annie"',
       'Sloper, Mr. William Thompson', 'Palsson, Miss. Torborg Danira',
       'Asplund, Mrs. Carl Oscar 

In [31]:
data_train.Ticket.unique()

array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649', '239865', '248698', '330923', '113788',
       '347077', '2631', '19950', '330959', '349216', 'PC 17601',
       'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677',
       'A./5. 2152', '345764', '2651', '7546', '11668', '349253',
       'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311',
       '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926',
       '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144',
       '2669', '113572', '36973', '347088', 'PC 17605', '2661',
       'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111',
       'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746',
       '248738', '364516', '345767', '345779', '330932', '113059',
       'SO/C 14885', '31012

In [32]:
data_train.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [27]:
data_train["Cabin"].str[:2]

0      NaN
1       C8
2      NaN
3       C1
4      NaN
      ... 
886    NaN
887     B4
888    NaN
889     C1
890    NaN
Name: Cabin, Length: 891, dtype: object